<a href="https://colab.research.google.com/github/larissavaladao/py6s_harmonize_sample/blob/main/2.deglint_sampling/deglint_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import packages

In [1]:
#import packages used
import ee
import pandas as pd
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import math
import geemap
import os
import math

In [2]:
#authenticate and initialize google earth engine (also necessary for geemap)
ee.Authenticate()
ee.Initialize(project='ee-curuai') #change project name here

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Curuai dataset

In [30]:
#function to insert a property with date of the point - Gee date format
def insert_date (feat):
    return feat.set('system:time_start',ee.Date.parse('YYYY-MM-dd HH:mm:ss',feat.get('DATE')))

In [31]:
#apply date function - new system:time_start property
roi_points = ee.FeatureCollection('projects/ee-curuai/assets/curuai_points').map(insert_date)
print(roi_points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-55.455998192971315, -2.26245869249737]}, 'id': '000000000000000000ea', 'properties': {'DATE': '2014-11-30 00:00:00', 'ID': 234, 'LATITUDE': '-2.26246', 'LONGITUDE': '-55.456', 'MISSION': 'IV', 'system:time_start': {'type': 'Date', 'value': 1417305600000}}}


In [32]:
# #create a polygon around the floodpalin area - roi
# roi_poly = ee.FeatureCollection('projects/ee-curuai/assets/roi_poly_curuai')

## Define date ranges for filtering images

In [33]:
advance = 16
# get the first and last dates os sample collection in the mission and extend the range by 16 days (Landsat interval between images)
startDate = ee.Date(roi_points.sort('system:time_start').first().get('system:time_start')).advance(-advance, 'day')
endDate = ee.Date(roi_points.sort('system:time_start',False).first().get('system:time_start')).advance(advance, 'day')

print('Data inicial de coleta: ',startDate.format().getInfo())
print('Data final de coleta: ',endDate.format().getInfo())

Data inicial de coleta:  2013-02-20T00:00:00
Data final de coleta:  2017-10-05T00:00:00


# Import image collections corrected to BOA

##Landsat 7 BOA

importar coleção com correção Py6S

In [34]:
ld7_path = 'projects/ee-curuai/assets/Py6S/LD7/ld7_py6s'
# print(ld7_path)
landsat7 = ee.ImageCollection(ld7_path)
landsat7.limit(5)

##Landsat 8 BOA

importar coleção com correção Py6S

In [35]:
ld8_path = 'projects/ee-curuai/assets/Py6S/LD8/ld8_py6s'
landsat8 = ee.ImageCollection(ld8_path)
landsat8.limit(5)

##Sentinel 2 BOA

importar coleção com correção Py6S

In [36]:
s2_path = 'projects/ee-curuai/assets/Py6S/S2/s2_py6s'
sentinel2 = ee.ImageCollection(s2_path).map(lambda img: img.set('system:time_start',ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filter(ee.Filter.eq('GRANULE_ID',img.get('GRANULE_ID'))).first().get('system:time_start')))
sentinel2.limit(5)

## Padronize band names

In [37]:
name_bands = ['blue','green','red','nir','swir1','swir2']

###Landsat 7

In [38]:
#renomear bandas
ld7 = landsat7.map(lambda img: img.select(['B1','B2','B3','B4','B5','B7'])\
                        .rename(name_bands))

### Landsat 8

In [39]:
ld8 = landsat8.map(lambda img: img.select(['B2','B3','B4','B5','B6','B7'])\
                        .rename(name_bands))

### Sentinel 2

In [40]:
s2 = sentinel2.map(lambda img: img.select(['B2','B3','B4','B8','B11','B12'])\
                        .rename(name_bands))


In [41]:
ld8.first()

# BRDF ajustment

## parameters

In [42]:
PI = ee.Number(math.pi);
MAX_SATELLITE_ZENITH = 7.5
MAX_DISTANCE = 1000000
UPPER_LEFT = 0
LOWER_LEFT = 1
LOWER_RIGHT = 2
UPPER_RIGHT = 3

##function

In [43]:
'''
Creates a collection of mosaics with a given temporal interval.
collection - the collection from which to make composites.
start - the date of the first composite (either a string or an ee.Date)
count - the number of composites to make
interval - The time between composites, in units of "units".
units - The units of step (day, week, month, year; see ee ee.Date.advance)
'''
def brdf (collection):

  def line_from_coords(coordinates, fromIndex, toIndex):
    '''
    helper function
    '''
    return ee.Geometry.LineString(ee.List([
        coordinates.get(fromIndex),
        coordinates.get(toIndex)]))

  def where(condition, trueValue, falseValue):
    trueMasked = trueValue.mask(condition)
    falseMasked = falseValue.mask(invertMask(condition))
    return trueMasked.unmask(falseMasked)


  def invertMask(mask):
    return mask.multiply(-1).add(1)


  def value(list,index):
    return ee.Number(list.get(index))


  def _kvol(sunAz, sunZen, viewAz, viewZen):
    """
    Calculate kvol kernel.
			From Lucht et al. 2000
			Phase angle = cos(solar zenith) cos(view zenith) + sin(solar zenith) sin(view zenith) cos(relative azimuth)
      calculate kvol and kvol0
      sunAZ
      sunZen
      viewAz
      viewZen
    """

    relative_azimuth = sunAz.subtract(viewAz).rename(['relAz'])
    pa1 = viewZen.cos().multiply(sunZen.cos())
    pa2 = viewZen.sin().multiply(sunZen.sin()).multiply(relative_azimuth.cos())
    phase_angle1 = pa1.add(pa2)
    phase_angle = phase_angle1.acos()
    p1 = ee.Image(PI.divide(2)).subtract(phase_angle)
    p2 = p1.multiply(phase_angle1)
    p3 = p2.add(phase_angle.sin())
    p4 = sunZen.cos().add(viewZen.cos())
    p5 = ee.Image(PI.divide(4))

    kvol = p3.divide(p4).subtract(p5).rename(['kvol'])

    viewZen0 = ee.Image(0)
    pa10 = viewZen0.cos().multiply(sunZen.cos())
    pa20 = viewZen0.sin().multiply(sunZen.sin()).multiply(relative_azimuth.cos())
    phase_angle10 = pa10.add(pa20)
    phase_angle0 = phase_angle10.acos()
    p10 = ee.Image(PI.divide(2)).subtract(phase_angle0)
    p20 = p10.multiply(phase_angle10)
    p30 = p20.add(phase_angle0.sin())
    p40 = sunZen.cos().add(viewZen0.cos())
    p50 = ee.Image(PI.divide(4))

    kvol0 = p30.divide(p40).subtract(p50).rename(['kvol0'])

    return [kvol, kvol0]

  def _correct_band(image, band_name, kvol, kvol0, f_iso, f_geo, f_vol):
    '''
    correct band function
    image : the image to apply the function to
    band_name
    kvol
    kvol0
    f_iso
    f_geo
    f_vol
    fiso + fvol * kvol + fgeo * kgeo
    '''
    iso = ee.Image(f_iso)
    geo = ee.Image(f_geo)
    vol = ee.Image(f_vol)
    pred = vol.multiply(kvol).add(geo.multiply(kvol)).add(iso).rename(['pred'])
    pred0 = vol.multiply(kvol0).add(geo.multiply(kvol0)).add(iso).rename(['pred0'])
    cfac = pred0.divide(pred).rename(['cfac'])
    corr = image.select(band_name).multiply(cfac).rename([band_name])
    return corr

  def _apply(image, kvol, kvol0):
    '''
    apply function to all bands
    http://www.mdpi.com/2072-4292/9/12/1325/htm#sec3dot2-remotesensing-09-0
    https://www.sciencedirect.com/science/article/pii/S0034425717302791
    image : the image to apply the functi
    kvol0
    '''
    f_iso = 0
    f_geo = 0
    f_vol = 0
    blue = _correct_band(image, 'blue', kvol, kvol0, f_iso=0.0774, f_geo=0.0079, f_vol=0.0372)
    green = _correct_band(image, 'green', kvol, kvol0, f_iso=0.1306, f_geo=0.0178, f_vol=0.0580)
    red = _correct_band(image, 'red', kvol, kvol0, f_iso=0.1690, f_geo=0.0227, f_vol=0.0574)
    re1 = _correct_band(image, 're1', kvol, kvol0, f_iso=0.2085, f_geo=0.0256, f_vol=0.0845)
    re2 = _correct_band(image, 're2', kvol, kvol0, f_iso=0.2316, f_geo=0.0273, f_vol=0.1003)
    re3 = _correct_band(image, 're3', kvol, kvol0, f_iso=0.2599, f_geo=0.0294, f_vol=0.1197)
    nir = _correct_band(image, 'nir', kvol, kvol0, f_iso=0.3093, f_geo=0.0330, f_vol=0.1535)
    re4 = _correct_band(image, 're4', kvol, kvol0, f_iso=0.2907, f_geo=0.0410, f_vol=0.1611)
    swir1 = _correct_band(image, 'swir1', kvol, kvol0, f_iso=0.3430, f_geo=0.0453, f_vol=0.1154);
    swir2 = _correct_band(image, 'swir2', kvol, kvol0, f_iso=0.2658, f_geo=0.0387, f_vol=0.0639)
    return image.select([]).addBands([blue, green, red, nir,swir1, swir2])

  def zenith(footprint):
    '''
    Get zenith.
    footprint: geometry of the image
    '''
    leftLine = line_from_coords(footprint, UPPER_LEFT, LOWER_LEFT)
    rightLine = line_from_coords(footprint, UPPER_RIGHT, LOWER_RIGHT)
    leftDistance = ee.FeatureCollection(leftLine).distance(MAX_DISTANCE)
    rightDistance = ee.FeatureCollection(rightLine).distance(MAX_DISTANCE)
    viewZenith = rightDistance.multiply(ee.Number(MAX_SATELLITE_ZENITH * 2))\
          .divide(rightDistance.add(leftDistance)) \
          .subtract(ee.Number(MAX_SATELLITE_ZENITH)) \
          .clip(ee.Geometry.Polygon(footprint)) \
          .rename(['viewZen'])
    return viewZenith.multiply(PI.divide(180))

  def azimuth(footprint):
    '''
    Get azimuth.
    footprint: geometry of the image
    '''

    def x(point):
      return ee.Number(ee.List(point).get(0))
    def y(point):
      return ee.Number(ee.List(point).get(1))

    upperCenter = line_from_coords(footprint, UPPER_LEFT, UPPER_RIGHT).centroid().coordinates()
    lowerCenter = line_from_coords(footprint, LOWER_LEFT, LOWER_RIGHT).centroid().coordinates()
    slope = ((y(lowerCenter)).subtract(y(upperCenter))).divide((x(lowerCenter)).subtract(x(upperCenter)))
    slopePerp = ee.Number(-1).divide(slope)
    azimuthLeft = ee.Image(PI.divide(2).subtract((slopePerp).atan()))
    return azimuthLeft.rename(['viewAz'])

  def getsunAngles(date, footprint):
    jdp = date.getFraction('year')
    seconds_in_hour = 3600
    hourGMT = ee.Number(date.getRelative('second', 'day')).divide(seconds_in_hour)

    latRad = ee.Image.pixelLonLat().select('latitude').multiply(PI.divide(180))
    longDeg = ee.Image.pixelLonLat().select('longitude')

    # // Julian day proportion in radians
    jdpr = jdp.multiply(PI).multiply(2)

    a = ee.List([0.000075, 0.001868, 0.032077, 0.014615, 0.040849])
    meanSolarTime = longDeg.divide(15.0).add(ee.Number(hourGMT))
    localSolarDiff1 = value(a, 0)\
            .add(value(a, 1).multiply(jdpr.cos())) \
            .subtract(value(a, 2).multiply(jdpr.sin())) \
            .subtract(value(a, 3).multiply(jdpr.multiply(2).cos())) \
            .subtract(value(a, 4).multiply(jdpr.multiply(2).sin()))

    localSolarDiff2 = localSolarDiff1.multiply(12 * 60)

    localSolarDiff = localSolarDiff2.divide(PI)
    trueSolarTime = meanSolarTime \
            .add(localSolarDiff.divide(60)) \
            .subtract(12.0)

    # // Hour as an angle;
    ah = trueSolarTime.multiply(ee.Number(MAX_SATELLITE_ZENITH * 2).multiply(PI.divide(180)))
    b = ee.List([0.006918, 0.399912, 0.070257, 0.006758, 0.000907, 0.002697, 0.001480])
    delta = value(b, 0) \
          .subtract(value(b, 1).multiply(jdpr.cos())) \
          .add(value(b, 2).multiply(jdpr.sin())) \
          .subtract(value(b, 3).multiply(jdpr.multiply(2).cos())) \
          .add(value(b, 4).multiply(jdpr.multiply(2).sin())) \
          .subtract(value(b, 5).multiply(jdpr.multiply(3).cos())) \
          .add(value(b, 6).multiply(jdpr.multiply(3).sin()))

    cosSunZen = latRad.sin().multiply(delta.sin()) \
          .add(latRad.cos().multiply(ah.cos()).multiply(delta.cos()))
    sunZen = cosSunZen.acos()

    # // sun azimuth from south, turning west
    sinSunAzSW = ah.sin().multiply(delta.cos()).divide(sunZen.sin())
    sinSunAzSW = sinSunAzSW.clamp(-1.0, 1.0)

    cosSunAzSW = (latRad.cos().multiply(-1).multiply(delta.sin())\
                    .add(latRad.sin().multiply(delta.cos()).multiply(ah.cos()))) \
                    .divide(sunZen.sin())
    sunAzSW = sinSunAzSW.asin()

    sunAzSW = where(cosSunAzSW.lte(0), sunAzSW.multiply(-1).add(PI), sunAzSW)
    sunAzSW = where(cosSunAzSW.gt(0).And(sinSunAzSW.lte(0)), sunAzSW.add(PI.multiply(2)), sunAzSW)

    sunAz = sunAzSW.add(PI)
    # Keep within [0, 2pi] range
    sunAz = where(sunAz.gt(PI.multiply(2)), sunAz.subtract(PI.multiply(2)), sunAz)

    footprint_polygon = ee.Geometry.Polygon(footprint)
    sunAz = sunAz.clip(footprint_polygon)
    sunAz = sunAz.rename(['sunAz'])
    sunZen = sunZen.clip(footprint_polygon).rename(['sunZen'])

    return [sunAz, sunZen]

  def applyBRDF(image):
    date = image.date()
    footprint = ee.List(image.geometry().bounds().bounds().coordinates().get(0))
    angles =  getsunAngles(date, footprint)
    sunAz = angles[0]
    sunZen = angles[1]

    viewAz = azimuth(footprint)
    viewZen = zenith(footprint)


    kval = _kvol(sunAz, sunZen, viewAz, viewZen)
    kvol = kval[0]
    kvol0 = kval[1]
    result = _apply(image, kvol.multiply(PI), kvol0.multiply(PI))

    return result

  collection = collection.map(applyBRDF)
  return collection;


##apply correction

In [44]:
brdf_ld7 = brdf(ld7)
brdf_ld7.limit(2)

In [45]:
brdf_ld8 = brdf(ld8)
brdf_ld8.limit(2)

In [46]:
brdf_s2 = brdf(s2)
brdf_s2.limit(2)

# Landsat band adjustment

In [55]:
def band_adjustmentLD8(img):
  blue = img.select('blue').multiply(1.0946).subtract(0.0107)
  green = img.select('green').multiply(1.0043).add(0.0026)
  red = img.select('red').multiply(1.0524).subtract(0.0015)
  nir=img.select('nir').multiply(0.8954).add(0.0033)
  swir1 = img.select('swir1').multiply(1.0049).add(0.0065)
  swir2 = img.select('swir2').multiply(1.0002).add(0.0046)

  return blue.addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2).copyProperties(img,['system:time_start','CLOUD_COVER',"system:index","LANDSAT_PRODUCT_ID"])

In [56]:
adj_ld8 = brdf_ld8.map(band_adjustmentLD8)
adj_ld8.limit(2)

In [57]:
def band_adjustmentLD7(img):
  blue = img.select('blue').multiply(1.1060).subtract(0.0139)
  green = img.select('green').multiply(0.9909).add(0.0041)
  red = img.select('red').multiply(1.0568).subtract(0.0024)
  nir=img.select('nir').multiply(1.0045).subtract(0.0076)
  swir1 = img.select('swir1').multiply(1.0361).add(0.0041)
  swir2 = img.select('swir2').multiply(1.0401).add(0.0086)

  return blue.addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2).copyProperties(img,['system:time_start','CLOUD_COVER',"system:index","LANDSAT_PRODUCT_ID"])

In [58]:
adj_ld7 = brdf_ld7.map(band_adjustmentLD7)
adj_ld7.limit(2)

# Transform into remote sensing  reflectance and sunglint correction

In [59]:
#dividir imagem corrigida por pi
# Rrs_sat_ac = Rsat_ac / pi

#fazer deglint
#Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
#correção no artigo INPE CURUAI
def deglint (img):
    Rrs = img.divide(math.pi)
    deglint = Rrs.select(['blue','green','red','nir'])\
    .subtract(Rrs.select('swir1'))

    return (deglint.updateMask(deglint.select(['red']).gt(0))
    .copyProperties(img,['system:time_start','CLOUD_COVER',"system:index","LANDSAT_PRODUCT_ID","GRANULE_ID","CLOUDY_PIXEL_PERCENTAGE"]))


### Landsat 7

In [60]:
landsat7_rs = adj_ld7.map(deglint).map(lambda img : img.set("SATELLITE_PRODUCT_ID",img.get("LANDSAT_PRODUCT_ID")))

landsat7_rs.limit(2)

### Landsat 8

In [61]:
landsat8_rs = adj_ld8.map(deglint).map(lambda img : img.set("SATELLITE_PRODUCT_ID",img.get("LANDSAT_PRODUCT_ID")))
landsat8_rs.limit(2)

### Sentinel 2

In [62]:
sentinel2_rs = brdf_s2.map(deglint).map(lambda img : img.set({"SATELLITE_PRODUCT_ID":img.get("GRANULE_ID"),
                                                          "CLOUD_COVER":img.get("CLOUDY_PIXEL_PERCENTAGE")}))

sentinel2_rs.limit(2)

In [63]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(roi_points,10)
vis =  {'bands':['red','green','blue'],'min':0.01,'max':0.06}
Map.addLayer(landsat7_rs.sort('CLOUD_COVER').first(),vis, 'landsat7')
Map.addLayer(landsat8_rs.sort('CLOUD_COVER').first(),vis, 'landsat8')
Map.addLayer(sentinel2_rs.first(), vis, 'sentinel2')

Map.addLayer(roi_points, {'color':'darkred'}, 'Data Points');
# Map.addLayer(roi_poly, {'color':'darkred'}, 'Data polygon');
Map

Map(center=[-2.184599717160098, -55.49304633852027], controls=(WidgetControl(options=['position', 'transparent…

# Sample data points pixel values

filter images that fall within a 16-day window period from each field point date

In [ ]:
def imgs_points(collection):
    def wrap(feat):
        date_point = ee.Date(feat.get('system:time_start'))
        data1 = date_point.advance(-16,'day')\
        .format('yyyy-MM-dd')

        data2 = date_point.advance(16,'day')\
        .format('yyyy-MM-dd')

        filtro = collection.filterDate(data1,data2)\
        .filterBounds(feat.geometry())\
        .map(lambda img: img.set({
            'dif_date_point':ee.Date(img.get('system:time_start')).difference(date_point, 'day')})\
            .copyProperties(feat,['ID']))


        return ee.ImageCollection(filtro.limit(10))

    return wrap

obtain statistics for the same location of the field point - with a 3 pixel window - and filter out if more than 4 pixels are masked   

In [ ]:
def sample_point(img):
  feat = roi_points.filter(ee.Filter.eq('ID',img.get('ID'))).first()
  geom = feat.geometry().buffer(45).bounds()

  mean = img.reduceRegion(geometry=geom,
                          scale=prj.nominalScale(),
                          crs=prj.crs(),
                          reducer=ee.Reducer.mean())
  median = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.median())
  minMax = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.minMax())
  count = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.count())
  return feat.set({
      "SATELLITE_PRODUCT_ID": img.get('SATELLITE_PRODUCT_ID'),
      'CLOUD_COVER':img.get('CLOUD_COVER'),
      'img_date':ee.Date(img.get('system:time_start')).format(),
      'dif_date_point': img.get('dif_date_point'),
      'blue_mean':mean.get('blue'),
      'green_mean':mean.get('green'),
      'red_mean':mean.get('red'),
      'nir_mean':mean.get('nir'),
      'blue_median':median.get('blue'),
      'green_meadin':median.get('green'),
      'red_median':median.get('red'),
      'nir_median':median.get('nir'),
      'blue_min':minMax.get('blue_min'),
      'green_min':minMax.get('green_min'),
      'red_min':minMax.get('red_min'),
      'nir_min':minMax.get('nir_min'),
      'blue_max':minMax.get('blue_max'),
      'green_max':minMax.get('green_max'),
      'red_max':minMax.get('red_max'),
      'nir_max':minMax.get('nir_max'),
      "count_pixel":count.get('red')
      })

filter_count = ee.Filter.gt('count_pixel',3)




### Landsat 7

In [ ]:
img_pointsLD7 = ee.ImageCollection(roi_points.map(imgs_points(landsat7_rs)).flatten().toList(2000))

In [ ]:
img_pointsLD7.size()


In [ ]:
img_pointsLD7.aggregate_count_distinct('ID')

KeyboardInterrupt: 

In [ ]:
img_pointsLD7.limit(5)

In [ ]:
reduced_LD7 = ee.FeatureCollection(img_pointsLD7.map(sample_point)).filter(filter_count)

In [ ]:
reduced_LD7.limit(2)

In [ ]:
reduced_LD7.size()

In [ ]:
reduced_LD7.aggregate_count_distinct('ID')

## Landsat 8

In [ ]:
img_pointsLD8 = ee.ImageCollection(roi_points.map(imgs_points(landsat8_rs)).flatten().toList(2000))

In [ ]:
img_pointsLD8.size()


In [ ]:
img_pointsLD8.aggregate_count_distinct('ID')

In [ ]:
img_pointsLD8.limit(5)

In [ ]:
reduced_LD8 = ee.FeatureCollection(img_pointsLD8.map(sample_point)).filter(filter_count)

In [ ]:
reduced_LD8.size()


In [ ]:
reduced_LD8.aggregate_count_distinct('ID')

In [ ]:
reduced_LD8.limit(2)

##Sentinel 2

In [ ]:
img_pointsS2 = ee.ImageCollection(roi_points.map(imgs_points(sentinel2_rs)).flatten().toList(2000))

In [ ]:
img_pointsS2.size()

In [ ]:
img_pointsS2.aggregate_count_distinct('ID')

In [ ]:
reduced_S2 = ee.FeatureCollection(img_pointsS2.map(sample_point)).filter(filter_count)

In [ ]:
reduced_S2.size()


In [ ]:
reduced_S2.aggregate_count_distinct('ID')

In [ ]:
reduced_S2.limit(2)

## Visualize sampled image by defining field point ID

In [ ]:
Map = geemap.Map()
Map.centerObject(roi_points,10)
id = 302
vis_params={'bands':['red','green','blue'],'min':0,'max':0.05}
Map.addLayer(img_pointsLD7.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'landsat7')
Map.addLayer(img_pointsLD8.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'landsat8')
Map.addLayer(img_pointsS2.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'sentinel2')

Map.addLayer(roi_points.filter(ee.Filter.eq('ID',id)), {'color':'darkred'}, 'Data Points');
Map

# Export data as CSV file

In [ ]:
#Exportar tabelas
#LD7
geemap.ee_to_csv(reduced_LD7, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD7_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/V_LD7_ACOLLITE_data.csv


In [ ]:
geemap.ee_to_csv(reduced_LD8, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD8_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/V_LD8_ACOLLITE_data.csv


In [ ]:
geemap.ee_to_csv(reduced_S2, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD7_ACOLLITE_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/VI_S2_DOS_data.csv


CORREÇÂO ATM:


1.   ACOLITE https://github.com/acolite/acolite
2.   6S com imagem da Dinha e parametros do artigo... talvez seja necessaria mais uma correção para isso



